# Run on Google Colab

https://colab.research.google.com/github/Kapil3003farmonaut/Api/blob/main/api_endpoints.ipynb

#### Note

If you are running the endpoints in Postman - they should work but sometimes have issue with postman webapp

You need to add following Headers

Application-Encoding : application/gzip

Content-Type : application/json


# Start


In [ ]:
import requests
import pandas as pd

UID = "APIKEY"  # Use your correct API key for testing

# Check Usage Details


In [15]:
def get_my_usage(UID):

    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getMyUsage"
    bodyObj = {
        "UID": UID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


response = get_my_usage(UID)

json_data = {}
try:
    json_data = response.json()
    if len(json_data) > 0:
        print("JSON data received.")
        print(json_data)
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")

Status code:  200
JSON data received.
{'OrderedUnits': 1000000, 'UsedUnits': 233, 'remainingUnits': 999767}


## Add/Submit field


In [ ]:
def submit_field(UID, CropCode, FieldName, PaymentType, PointCoordinates):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/submitField"
    bodyObj = {
        "UID": UID,
        "CropCode": CropCode,
        "FieldName": FieldName,
        "PaymentType": PaymentType,
        "Points": PointCoordinates,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


CropCode = "1r"
FieldName = "My Test Field 24"
PaymentType = 1
PointCoordinates = [
    [78.47214764936771, 17.410292565471753],
    [78.47341901644077, 17.41025161622103],
    [78.47430414541569, 17.409652732380625],
    [78.47315615995731, 17.409586189610526],
]

response = submit_field(UID, CropCode, FieldName, PaymentType, PointCoordinates)

json_data = {}
try:
    json_data = response.json()
    if len(json_data) > 0:

        print("JSON data received.")

        print("Field ID - ", json_data["FieldID"])
        print(json_data)  ##["FieldID"
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")

# Get all field IDs


In [ ]:
def get_all_feildids_(UID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getAllFieldIDs"
    bodyObj = {
        "UID": UID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response

# Get All Fields/Farmers Data


In [ ]:
def get_all_feilds_data(UID):
    endpointUrl = (
        "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getAllFarmersData"
    )
    bodyObj = {
        "UID": UID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


all_fields_data = get_all_feilds_data(UID)

json_data = {}
df_field_data = pd.DataFrame()

try:
    json_data = all_fields_data.json()

    if len(json_data) > 0:

        print("JSON data received.")
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")

if len(json_data) > 0:
    data_all_Field = []
    for fieldId, fieldDetails in all_fields_data.json().items():
        # print(fieldId)
        lat_lon_pairs = []

        # Define custom sorting order
        def custom_sort(key):
            if key == "a":
                return 0
            elif key.startswith("P_"):
                return int(key[2:])
            else:
                return float("inf")  # Place any other keys at the end

        # Sort the keys of the Coordinates dictionary using the custom sorting function
        sorted_coordinates = sorted(fieldDetails["Coordinates"].keys(), key=custom_sort)

        # Iterate over the sorted keys
        for point in sorted_coordinates:
            coordinates = fieldDetails["Coordinates"][point]
            latitude = float(coordinates["Latitude"])
            longitude = float(coordinates["Longitude"])
            lat_lon_pairs.append([latitude, longitude])

        # for point, coordinates in fieldDetails["Coordinates"].items():
        #     latitude = float(coordinates["Latitude"])
        #     longitude = float(coordinates["Longitude"])
        #     lat_lon_pairs.append([latitude, longitude])

        data_all_Field.append(
            {
                "FieldID": fieldId,
                # "FieldAddress": fieldDetails["FieldAddress"],
                "FieldArea": fieldDetails["FieldArea"],
                "hUnits": fieldDetails["hUnits"],
                "FieldMaxLat": fieldDetails["FieldMaxLat"],
                "FieldMaxLong": fieldDetails["FieldMaxLong"],
                "FieldMinLat": fieldDetails["FieldMinLat"],
                "FieldMinLong": fieldDetails["FieldMinLong"],
                "coodfinates": lat_lon_pairs,
            }
        )

    df_field_data = pd.DataFrame(data_all_Field)

df_field_data

# Delete Field


In [ ]:
from json.decoder import JSONDecodeError


def delete_field(UID, FieldID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/deleteField"
    bodyObj = {"UID": UID, "FieldID": FieldID}

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response

    # try:
    #     json_data = response.json()
    #     print(json_data)
    # except (AttributeError, JSONDecodeError) as e:
    #     print("Error occurred:", e)


FieldID = "1718697978627"
response = delete_field(UID, FieldID)

# Get Single Fields Data


In [ ]:
def get_single_field_data(UID, FieldID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getFarmerData"
    bodyObj = {"UID": UID, "FieldID": FieldID}

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


FieldID = "1718698527459"
response = get_single_field_data(UID, FieldID)

json_data = {}
try:
    json_data = response.json()
    if len(json_data) > 0:

        print("JSON data received.")
        print(json_data)  ##["FieldID"
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")

# Sensed Days - Find when satellite data is available :dates


In [ ]:
def get_sesnsed_days(UID, FieldID):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getSensedDays"
    bodyObj = {
        "UID": UID,
        "FieldID": FieldID,
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


FieldID = "1718698527459"
response = get_sesnsed_days(UID, FieldID)

json_data = {}
try:
    json_data = response.json()

    if len(json_data) > 0:
        print("JSON data received.")
        print(json_data)  ##["FieldID"
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")

# Get Sat Image / Image URl


In [ ]:
def get_sat_image(UID, fieldId, SensedDay, ImageType):

    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getFieldImage"
    bodyObj = {
        "UID": UID,
        "FieldID": fieldId,
        "ImageType": ImageType,
        "SensedDay": SensedDay,
        "Colormap": "1",
    }

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response


FieldID = "1718698527459"
SensedDay = "20240616"
ImageType = "ndvi"  # keep it lowercase
response = get_sat_image(UID, FieldID, SensedDay, ImageType)

json_data = {}
try:
    json_data = response.json()

    if len(json_data) > 0:
        print("JSON data received.")
        print(json_data)  ##["FieldID"
        # Further processing of the JSON data can be done here
    else:
        print("No JSON data received.")
except ValueError:
    print("Invalid JSON data received or no JSON data present.")

## Get Field Area by boundary points


In [ ]:
def get_area_by_boundary_points(UID, coordinates):
    endpointUrl = "https://us-central1-farmbase-b2f7e.cloudfunctions.net/getFieldAreaByBoundaryPoints"
    bodyObj = {"UID": UID, "Points": coordinates}

    response = requests.post(endpointUrl, json=bodyObj)

    print("Status code: ", response.status_code)

    return response